In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tqdm import tqdm
import numpy as np 
from scipy.stats import norm

import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.append("..")
from src.py.option import AsianOption
from src.py.tasmanian_tools import make_grid

In [3]:
s_0_l=80.0
s_0_r=120.0
sigma_l=0.1
sigma_r=0.2
mu_l=0.02
mu_r=0.05
T_l=0.9
T_r=1.0
K_l=109.0
K_r=110.0

In [4]:
# max possible dim, level before 16 GB RAM is full: 40, 4
dim = 16
aop = AsianOption(d=dim)
level = 4
lb, rb = np.zeros(dim), np.ones(dim)

grid = make_grid(dim, level, lb, rb, "gauss-patterson")
weights, points = grid.getQuadratureWeights(), grid.getPoints()
pre_factor = np.exp(-aop.r*aop.T+aop.M)
payout_v = aop.payout_func_opt(points)
pre_factor*np.inner(weights, payout_v), aop.scholes_call()

(7.152087238522643, 7.15590176686306)

In [5]:
aop.delta_t == 1/dim

True

In [6]:
s_0 = np.linspace(s_0_l, s_0_r, 21)
sigma = np.linspace(sigma_l, sigma_r, 5)
mu = np.linspace(mu_l, mu_r, 9)
T = np.linspace(T_l, T_r, 9)
K = np.linspace(K_l, K_r, 9)

In [7]:
mesh = np.meshgrid(s_0, sigma, mu, T, K)
grid_ar = np.stack(mesh, -1).reshape(-1, 5)

## vectorizing training data generation takes up too much memory
# -> make a loop

#for s_0, sigma, mu, T, K in grid_ar[:10]:
#    print(s_0, sigma, mu, T, K)


In [ ]:
#grid_ar.shape[0]
Y = np.empty((100, 2), dtype=np.float64)
i = 0
for s_0, sigma, mu, T, K in grid_ar[:100]:
    aop = AsianOption(S_0=s_0, sigma=sigma, r=mu, T=T, K=K, d=dim)
    pre_factor = np.exp(-aop.r*aop.T+aop.M)
    payout_v = aop.payout_func_opt(points)
    Y[i] = pre_factor*np.inner(weights, payout_v), aop.scholes_call()
    i=+1

In [ ]:
ar = np.load("sparse_geom_asian_out.npy")
ar[990:1000]

In [10]:
#plt.scatter(range(10), np.abs(np.diff(ar[990:1000])))
print(grid_ar[999])
op = AsianOption(
    S_0=grid_ar[999, 0],
    sigma=grid_ar[999, 1],
    r=grid_ar[999, 2], T=grid_ar[999, 3], K=grid_ar[999, 4], d=dim)
op.scholes_call()

[8.200e+01 1.000e-01 3.125e-02 9.375e-01 1.090e+02]


4.1200089200465264e-07